In [ ]:
# ising3d.py
import numpy as np


class Ising3D:
    """
    3D Ising model on an LxLxL cubic lattice with periodic boundary conditions.
    Spins S[i,j,k] = ±1.

    Parameters
    ----------
    L : int
        Linear system size (N = L^3 spins).
    T : float
        Temperature (k_B = 1, J = 1 by default).
    J : float
        Coupling constant (ferromagnetic if J > 0).
    h : float
        External magnetic field (default 0).
    rng : np.random.Generator or None
        Random number generator. If None, a new default_rng() is created.
    """

    def __init__(self, L, T, J=1.0, h=0.0, rng=None):
        self.L = int(L)
        self.N = self.L ** 3
        self.J = float(J)
        self.h = float(h)
        self.set_temperature(T)

        self.rng = rng if rng is not None else np.random.default_rng()

        # Spins as int8 to save memory, ±1
        self.spins = np.ones((self.L, self.L, self.L), dtype=np.int8)

    # ----------------------------
    # Basic configuration methods
    # ----------------------------
    def set_temperature(self, T):
        self.T = float(T)
        if self.T <= 0:
            raise ValueError("Temperature T must be positive.")
        self.beta = 1.0 / self.T

    def randomize_spins(self):
        """Random initial condition: spins = ±1 with equal probability."""
        self.spins = self.rng.choice([-1, 1], size=(self.L, self.L, self.L)).astype(np.int8)

    def set_all_up(self):
        """Ordered initial condition: all spins +1."""
        self.spins.fill(1)

    # ----------------------------
    # Energy and magnetisation
    # ----------------------------
    def magnetisation(self):
        """Total magnetisation M and magnetisation per spin m."""
        M = np.sum(self.spins, dtype=np.int64)
        m = M / self.N
        return M, m

    def _neighbor_sum(self, i, j, k):
        """Sum of nearest neighbours of spin at (i,j,k) with periodic boundaries."""
        L = self.L
        s = self.spins
        return (
            s[(i + 1) % L, j, k] +
            s[(i - 1) % L, j, k] +
            s[i, (j + 1) % L, k] +
            s[i, (j - 1) % L, k] +
            s[i, j, (k + 1) % L] +
            s[i, j, (k - 1) % L]
        )

    def local_energy(self, i, j, k):
        """
        Local energy contribution of spin at (i,j,k) including field:
        E_i = -J * S_i * sum_{nn} S_j - h * S_i
        """
        Si = self.spins[i, j, k]
        nn_sum = self._neighbor_sum(i, j, k)
        return -self.J * Si * nn_sum - self.h * Si

    def total_energy(self):
        """
        Total energy of the system.
        For efficiency, compute via nearest-neighbour bonds once.
        """
        s = self.spins
        J = self.J
        h = self.h

        # Periodic neighbour sums in +x, +y, +z directions to avoid double counting
        E_bonds = 0.0
        E_bonds += np.sum(s * np.roll(s, shift=-1, axis=0))  # x-direction
        E_bonds += np.sum(s * np.roll(s, shift=-1, axis=1))  # y-direction
        E_bonds += np.sum(s * np.roll(s, shift=-1, axis=2))  # z-direction

        E_bonds *= -J  # -J sum_{<ij>} S_i S_j

        # Field term: -h sum_i S_i
        E_field = -h * np.sum(s)

        return float(E_bonds + E_field)

    # ----------------------------
    # Metropolis single-spin update
    # ----------------------------
    def metropolis_sweep(self):
        """
        One Metropolis sweep = N attempted single-spin flips.

        Returns
        -------
        accept_rate : float
            Fraction of accepted spin flips during this sweep.
        """
        L = self.L
        s = self.spins
        beta = self.beta
        J = self.J
        h = self.h

        accepted = 0

        for _ in range(self.N):
            # Pick random site
            i = self.rng.integers(0, L)
            j = self.rng.integers(0, L)
            k = self.rng.integers(0, L)

            Si = s[i, j, k]
            nn_sum = self._neighbor_sum(i, j, k)

            # ΔE for flipping Si -> -Si:
            # before: E_i = -J Si nn_sum - h Si
            # after:  E_i' = -J (-Si) nn_sum - h (-Si) = J Si nn_sum + h Si
            # ΔE = E_i' - E_i = 2 J Si nn_sum + 2 h Si
            dE = 2.0 * Si * (J * nn_sum + h)

            if dE <= 0.0:
                s[i, j, k] = -Si
                accepted += 1
            else:
                if self.rng.random() < np.exp(-beta * dE):
                    s[i, j, k] = -Si
                    accepted += 1

        return accepted / self.N

    # ----------------------------
    # Wolff cluster update
    # ----------------------------
    def wolff_cluster_step(self):
        """
        Perform one Wolff cluster flip and return the cluster size.

        We build a cluster of like spins starting from a random seed and
        add neighbours with probability p_add = 1 - exp(-2 β J).
        """
        L = self.L
        s = self.spins
        beta = self.beta
        J = self.J

        # Bond-addition probability
        p_add = 1.0 - np.exp(-2.0 * beta * J)

        # Choose random seed spin
        i0 = self.rng.integers(0, L)
        j0 = self.rng.integers(0, L)
        k0 = self.rng.integers(0, L)

        spin_seed = s[i0, j0, k0]

        # Boolean visited mask to avoid re-adding sites
        visited = np.zeros_like(s, dtype=bool)

        # Stack / queue for cluster growth
        stack = [(i0, j0, k0)]
        visited[i0, j0, k0] = True

        cluster_sites = []

        while stack:
            i, j, k = stack.pop()
            cluster_sites.append((i, j, k))

            # Check all 6 neighbours
            neighbours = (
                ((i + 1) % L, j, k),
                ((i - 1) % L, j, k),
                (i, (j + 1) % L, k),
                (i, (j - 1) % L, k),
                (i, j, (k + 1) % L),
                (i, j, (k - 1) % L),
            )

            for ni, nj, nk in neighbours:
                if visited[ni, nj, nk]:
                    continue
                if s[ni, nj, nk] != spin_seed:
                    continue
                # Same spin as seed; consider adding to cluster
                if self.rng.random() < p_add:
                    visited[ni, nj, nk] = True
                    stack.append((ni, nj, nk))

        # Flip entire cluster
        for (i, j, k) in cluster_sites:
            s[i, j, k] = -s[i, j, k]

        cluster_size = len(cluster_sites)
        return cluster_size

    def wolff_effective_sweep(self):
        """
        Perform Wolff cluster flips until the total number of flipped spins
        is at least N. This defines one 'effective sweep' comparable to
        one Metropolis sweep.

        Returns
        -------
        n_clusters : int
            Number of clusters flipped.
        mean_cluster_size : float
            Mean cluster size during this effective sweep.
        """
        total_flipped = 0
        cluster_sizes = []

        while total_flipped < self.N:
            size = self.wolff_cluster_step()
            cluster_sizes.append(size)
            total_flipped += size

        n_clusters = len(cluster_sizes)
        mean_size = np.mean(cluster_sizes)
        return n_clusters, float(mean_size)


In [5]:
# run_simulation.py
import numpy as np


def run_simulation(
    L,
    T,
    algo="metropolis",
    J=1.0,
    h=0.0,
    n_equil_sweeps=2000,
    n_meas_sweeps=5000,
    meas_interval=10,
    seed=None,
):
    """
    Run a single simulation at fixed (L, T, J, h) with either Metropolis or Wolff.

    Returns
    -------
    results : dict
        Contains time series of observables and metadata:
        {
            "L": L,
            "T": T,
            "algo": algo,
            "sweeps": sweeps_array,
            "E": energies,
            "M": magnetisations,
            "m": magnetisation_per_spin,
            "accept_rate": accept_rates or None,
            "cluster_info": list of (n_clusters, mean_size) or None
        }
    """

    rng = np.random.default_rng(seed)

    model = Ising3D(L=L, T=T, J=J, h=h, rng=rng)
    # Choose initial condition; you can switch to set_all_up() if needed
    model.randomize_spins()

    energies = []
    mags = []
    mags_per_spin = []
    sweeps = []
    accept_rates = []
    cluster_stats = []

    # 1) Equilibration
    if algo.lower() == "metropolis":
        for _ in range(n_equil_sweeps):
            model.metropolis_sweep()

    elif algo.lower() == "wolff":
        for _ in range(n_equil_sweeps):
            model.wolff_effective_sweep()

    else:
        raise ValueError("algo must be 'metropolis' or 'wolff'.")

    # 2) Measurement phase
    sweep_counter = 0

    for meas_step in range(n_meas_sweeps):
        if algo.lower() == "metropolis":
            acc = model.metropolis_sweep()
            accept_rates.append(acc)
        else:
            n_clusters, mean_size = model.wolff_effective_sweep()
            cluster_stats.append((n_clusters, mean_size))

        sweep_counter += 1

        # Record measurements every meas_interval sweeps
        if (meas_step % meas_interval) == 0:
            E = model.total_energy()
            M, m = model.magnetisation()

            energies.append(E / model.N)  # energy per spin
            mags.append(M)
            mags_per_spin.append(m)
            sweeps.append(sweep_counter)

    results = {
        "L": L,
        "T": T,
        "J": J,
        "h": h,
        "algo": algo.lower(),
        "sweeps": np.array(sweeps),
        "E": np.array(energies),
        "M": np.array(mags),
        "m": np.array(mags_per_spin),
        "accept_rate": np.array(accept_rates) if algo.lower() == "metropolis" else None,
        "cluster_info": np.array(cluster_stats) if algo.lower() == "wolff" else None,
    }

    return results


if __name__ == "__main__":
    # Tiny smoke test run (this is not a real production run)
    L = 20
    T = 30

    res_met = run_simulation(L=L, T=T, algo="metropolis", n_equil_sweeps=500, n_meas_sweeps=500)
    res_wolff = run_simulation(L=L, T=T, algo="wolff", n_equil_sweeps=500, n_meas_sweeps=500)

    print("Metropolis: mean e =", np.mean(res_met["E"]), "mean |m| =", np.mean(np.abs(res_met["m"])))
    print("Wolff     : mean e =", np.mean(res_wolff["E"]), "mean |m| =", np.mean(np.abs(res_wolff["m"])))


Metropolis: mean e = -0.09913 mean |m| = 0.01019
Wolff     : mean e = -0.10129999999999999 mean |m| = 0.01031


In [6]:
# viz_ising3d.py
import numpy as np
import plotly.graph_objects as go


class Ising3DVisualizer:
    """
    Visualise 3D Ising spin configurations in interactive 3D using Plotly.

    Usage pattern:
    --------------
    vis = Ising3DVisualizer(L)
    vis.add_snapshot(spins_t0, time_label=0)
    vis.add_snapshot(spins_t1, time_label=100)
    fig = vis.plot_snapshot(1)        # show 2nd snapshot
    fig = vis.animate_snapshots()     # slider over all snapshots
    """

    def __init__(self, L, name="Ising 3D"):
        self.L = int(L)
        self.name = name
        self.snapshots = []   # list of 3D arrays (L, L, L)
        self.time_labels = [] # e.g. sweep numbers

    # ---------------------------
    # Data management
    # ---------------------------
    def add_snapshot(self, spins, time_label=None):
        """
        Store a copy of a spin configuration.

        Parameters
        ----------
        spins : array-like, shape (L, L, L)
            Spin configuration with values ±1.
        time_label : any (optional)
            Label for this snapshot (e.g. sweep number).
        """
        spins = np.asarray(spins)
        if spins.shape != (self.L, self.L, self.L):
            raise ValueError(f"spins must have shape {(self.L, self.L, self.L)}, got {spins.shape}")

        self.snapshots.append(spins.copy())
        self.time_labels.append(time_label)

    def __len__(self):
        return len(self.snapshots)

    # ---------------------------
    # Internal helper
    # ---------------------------
    def _prepare_scatter_data(self, spins, downsample=1):
        """
        Convert 3D spin array into x, y, z, color arrays for scatter3d.

        Parameters
        ----------
        spins : ndarray, shape (L, L, L)
        downsample : int
            Take every 'downsample'-th lattice point in each direction
            to reduce number of plotted points for large L.

        Returns
        -------
        x, y, z, colors : 1D arrays
        """
        L = self.L
        if downsample < 1:
            raise ValueError("downsample must be >= 1")

        # Indices to keep
        idx = np.arange(0, L, downsample)
        s_ds = spins[np.ix_(idx, idx, idx)]

        # Lattice coordinates
        X, Y, Z = np.meshgrid(idx, idx, idx, indexing="ij")
        X = X.flatten()
        Y = Y.flatten()
        Z = Z.flatten()
        S = s_ds.flatten()

        # Map spins to colors: +1 = one color, -1 = another
        # We'll use a simple two-color scheme via custom colorscale
        colors = S  # values -1 or +1, used with colorscale later

        return X, Y, Z, colors

    # ---------------------------
    # Single snapshot 3D plot
    # ---------------------------
    def plot_snapshot(self, snapshot_index=0, downsample=1, show=True):
        """
        Plot a single snapshot as 3D scatter.

        Parameters
        ----------
        snapshot_index : int
            Index of stored snapshot to plot (0-based).
        downsample : int
            Plot only every 'downsample'-th lattice point.
        show : bool
            If True, calls fig.show().

        Returns
        -------
        fig : plotly.graph_objects.Figure
        """
        if not self.snapshots:
            raise RuntimeError("No snapshots stored. Call add_snapshot(...) first.")

        if snapshot_index < 0 or snapshot_index >= len(self.snapshots):
            raise IndexError(f"snapshot_index {snapshot_index} out of range [0, {len(self.snapshots)-1}]")

        spins = self.snapshots[snapshot_index]
        time_label = self.time_labels[snapshot_index]

        x, y, z, c = self._prepare_scatter_data(spins, downsample=downsample)

        fig = go.Figure(
            data=[
                go.Scatter3d(
                    x=x,
                    y=y,
                    z=z,
                    mode="markers",
                    marker=dict(
                        size=4,  # adjust depending on L/downsample
                        opacity=0.8,
                        color=c,
                        colorscale=[
                            [0.0, "blue"],   # for -1
                            [0.5, "blue"],
                            [0.5, "red"],    # for +1
                            [1.0, "red"],
                        ],
                        cmin=-1,
                        cmax=1,
                    ),
                )
            ]
        )

        title = f"{self.name} - snapshot {snapshot_index}"
        if time_label is not None:
            title += f" (t = {time_label})"

        fig.update_layout(
            title=title,
            scene=dict(
                xaxis_title="i",
                yaxis_title="j",
                zaxis_title="k",
                aspectmode="cube",
            ),
            margin=dict(l=0, r=0, b=0, t=40),
        )

        if show:
            fig.show()

        return fig

    # ---------------------------
    # Multi-snapshot animation / slider
    # ---------------------------
    def animate_snapshots(self, indices=None, downsample=1, show=True):
        """
        Create an interactive 3D animation with a slider over snapshots.

        Parameters
        ----------
        indices : list[int] or None
            Which snapshot indices to include in the animation.
            If None, use all snapshots in order.
        downsample : int
            Plot only every 'downsample'-th lattice point.
        show : bool
            If True, calls fig.show().

        Returns
        -------
        fig : plotly.graph_objects.Figure
        """
        if not self.snapshots:
            raise RuntimeError("No snapshots stored. Call add_snapshot(...) first.")

        n_snaps = len(self.snapshots)
        if indices is None:
            indices = list(range(n_snaps))

        # Sanity check indices
        indices = list(indices)
        for idx in indices:
            if idx < 0 or idx >= n_snaps:
                raise IndexError(f"Snapshot index {idx} out of range [0, {n_snaps-1}]")

        # Build frames
        frames = []
        for frame_id, snap_idx in enumerate(indices):
            spins = self.snapshots[snap_idx]
            time_label = self.time_labels[snap_idx]
            x, y, z, c = self._prepare_scatter_data(spins, downsample=downsample)

            frame_name = f"frame_{frame_id}"

            scatter = go.Scatter3d(
                x=x,
                y=y,
                z=z,
                mode="markers",
                marker=dict(
                    size=4,
                    opacity=0.8,
                    color=c,
                    colorscale=[
                        [0.0, "blue"],
                        [0.5, "blue"],
                        [0.5, "red"],
                        [1.0, "red"],
                    ],
                    cmin=-1,
                    cmax=1,
                ),
            )

            title = f"{self.name} - snapshot {snap_idx}"
            if time_label is not None:
                title += f" (t = {time_label})"

            frames.append(go.Frame(data=[scatter], name=frame_name, layout=dict(title=title)))

        # Initial frame = first entry
        first_spins = self.snapshots[indices[0]]
        x0, y0, z0, c0 = self._prepare_scatter_data(first_spins, downsample=downsample)

        fig = go.Figure(
            data=[
                go.Scatter3d(
                    x=x0,
                    y=y0,
                    z=z0,
                    mode="markers",
                    marker=dict(
                        size=4,
                        opacity=0.8,
                        color=c0,
                        colorscale=[
                            [0.0, "blue"],
                            [0.5, "blue"],
                            [0.5, "red"],
                            [1.0, "red"],
                        ],
                        cmin=-1,
                        cmax=1,
                    ),
                )
            ],
            frames=frames,
        )

        # Slider steps
        slider_steps = []
        for frame_id, snap_idx in enumerate(indices):
            name = f"frame_{frame_id}"
            t_label = self.time_labels[snap_idx]
            step_label = f"idx {snap_idx}"
            if t_label is not None:
                step_label += f" (t={t_label})"

            slider_steps.append(
                dict(
                    method="animate",
                    args=[[name], {"mode": "immediate", "frame": {"duration": 0, "redraw": True}}],
                    label=step_label,
                )
            )

        sliders = [
            dict(
                active=0,
                currentvalue={"prefix": "Snapshot: "},
                pad={"t": 10},
                steps=slider_steps,
            )
        ]

        fig.update_layout(
            title=f"{self.name} - 3D magnetisation map animation",
            scene=dict(
                xaxis_title="i",
                yaxis_title="j",
                zaxis_title="k",
                aspectmode="cube",
            ),
            margin=dict(l=0, r=0, b=0, t=40),
            sliders=sliders,
            updatemenus=[
                dict(
                    type="buttons",
                    showactive=False,
                    x=0.05,
                    y=1.1,
                    xanchor="left",
                    yanchor="top",
                    buttons=[
                        dict(
                            label="Play",
                            method="animate",
                            args=[
                                None,
                                {
                                    "frame": {"duration": 500, "redraw": True},
                                    "fromcurrent": True,
                                    "transition": {"duration": 0},
                                },
                            ],
                        ),
                        dict(
                            label="Pause",
                            method="animate",
                            args=[
                                [None],
                                {
                                    "frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0},
                                },
                            ],
                        ),
                    ],
                )
            ],
        )

        if show:
            fig.show()

        return fig


In [ ]:
 # example_usage.py
import os
import csv
import numpy as np


L = 16
T = 4.5
J = 5.0
h = 0.0

model = Ising3D(L=L, T=T, J=J, h=h)
model.randomize_spins()

vis = Ising3DVisualizer(L=L, name="3D Ising demo")

n_equil_sweeps = 1000
n_meas_sweeps = 2000
store_every = 200  # store one snapshot every 200 sweeps

# --------- prepare output dirs / metadata ----------
output_dir = "snapshots"
os.makedirs(output_dir, exist_ok=True)

meta_rows = []  # we'll write this to snapshots_meta.csv at the end


def save_snapshot_to_csv(spins, sweep_label, snapshot_idx):
    """
    Save a 3D spin configuration to CSV as (i,j,k,spin).
    Also return the filename so we can log it in metadata.
    """
    # Build filename
    fname = f"snapshot_L{L}_T{T:.3f}_J{J:.3f}_h{h:.3f}_sweep{sweep_label}_idx{snapshot_idx:04d}.csv"
    fpath = os.path.join(output_dir, fname)

    # Create index grid and flatten
    Lloc = spins.shape[0]
    i, j, k = np.indices((Lloc, Lloc, Lloc))
    data = np.column_stack([i.ravel(), j.ravel(), k.ravel(), spins.ravel()])

    # Save CSV: i,j,k,spin
    np.savetxt(
        fpath,
        data,
        delimiter=",",
        header="i,j,k,spin",
        comments="",  # avoid '#' at start of header
        fmt="%d",
    )

    return fpath


# ---------------- Equilibration ----------------
for sweep in range(n_equil_sweeps):
    model.metropolis_sweep()

# ------------- Measurement + snapshots -------------
snapshot_counter = 0

for sweep in range(n_meas_sweeps):
    model.metropolis_sweep()

    if (sweep % store_every) == 0:
        # Time label = total sweeps since start (equil + meas)
        time_label = n_equil_sweeps + sweep

        # Store in visualizer for plotting
        vis.add_snapshot(model.spins, time_label=time_label)

        # Save to CSV
        snapshot_file = save_snapshot_to_csv(
            spins=model.spins,
            sweep_label=time_label,
            snapshot_idx=snapshot_counter,
        )

        # Record metadata row
        meta_rows.append({
            "snapshot_idx": snapshot_counter,
            "snapshot_file": snapshot_file,
            "sweep_label": time_label,
            "L": L,
            "T": T,
            "J": J,
            "h": h,
            "n_equil_sweeps": n_equil_sweeps,
            "n_meas_sweeps": n_meas_sweeps,
            "store_every": store_every,
        })

        snapshot_counter += 1

print(f"Stored {len(vis)} snapshots and CSV files")

# ------------- Write metadata CSV -------------
meta_path = os.path.join(output_dir, "snapshots_meta.csv")
with open(meta_path, "w", newline="") as f:
    fieldnames = [
        "snapshot_idx",
        "snapshot_file",
        "sweep_label",
        "L",
        "T",
        "J",
        "h",
        "n_equil_sweeps",
        "n_meas_sweeps",
        "store_every",
    ]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(meta_rows)

print(f"Metadata written to {meta_path}")

# ------------- Plotting -------------
# Plot a single snapshot (e.g. last one)
vis.plot_snapshot(snapshot_index=len(vis) - 1, downsample=1)

# Create an animation over all stored snapshots
vis.animate_snapshots(indices=None, downsample=1)



Stored 10 snapshots


In [16]:
# example_usage.py
import os
import csv
import numpy as np


# ----------------- Simulation parameters -----------------
L = 16
T = 4.5
J = 2.0
h = 0.0

n_equil_sweeps = 100
n_meas_sweeps = 500
store_every = 50  # store one snapshot every 200 sweeps

# ----------------- Create model & visualizer -----------------
model = Ising3D(L=L, T=T, J=J, h=h,)
model.randomize_spins()

vis = Ising3DVisualizer(L=L, name="3D Ising demo")

# ----------------- Run directory setup -----------------
base_runs_dir = "runs"
os.makedirs(base_runs_dir, exist_ok=True)

run_name = (
    f"run_L{L}"
    f"_T{T:.3f}"
    f"_J{J:.3f}"
    f"_h{h:.3f}"
    f"_eq{n_equil_sweeps}"
    f"_meas{n_meas_sweeps}"
    f"_freq{store_every}"
)

run_dir = os.path.join(base_runs_dir, run_name)
os.makedirs(run_dir, exist_ok=True)

print(f"Run directory: {run_dir}")

# This will hold metadata for all snapshots in THIS run
meta_rows = []


def save_snapshot_to_csv(spins, sweep_label, snapshot_idx):
    """
    Save a 3D spin configuration to CSV as (i,j,k,spin) inside this run's folder.
    """
    # Snapshot filename only needs sweep and index; run params are in the folder name
    fname = f"snapshot_sweep{sweep_label}_idx{snapshot_idx:04d}.csv"
    fpath = os.path.join(run_dir, fname)

    Lloc = spins.shape[0]
    i, j, k = np.indices((Lloc, Lloc, Lloc))
    data = np.column_stack([i.ravel(), j.ravel(), k.ravel(), spins.ravel()])

    np.savetxt(
        fpath,
        data,
        delimiter=",",
        header="i,j,k,spin",
        comments="",
        fmt="%d",
    )

    return fpath


# ----------------- Equilibration -----------------
for sweep in range(n_equil_sweeps):
    model.metropolis_sweep()

# ----------------- Measurement + snapshots -----------------
snapshot_counter = 0

for sweep in range(n_meas_sweeps):
    model.metropolis_sweep()

    if (sweep % store_every) == 0:
        time_label = n_equil_sweeps + sweep

        # store for visualisation
        vis.add_snapshot(model.spins, time_label=time_label)

        # save to CSV
        snapshot_file = save_snapshot_to_csv(
            spins=model.spins,
            sweep_label=time_label,
            snapshot_idx=snapshot_counter,
        )

        # metadata for this snapshot
        meta_rows.append({
            "snapshot_idx": snapshot_counter,
            "snapshot_file": os.path.basename(snapshot_file),
            "sweep_label": time_label,
            "L": L,
            "T": T,
            "J": J,
            "h": h,
            "n_equil_sweeps": n_equil_sweeps,
            "n_meas_sweeps": n_meas_sweeps,
            "store_every": store_every,
        })

        snapshot_counter += 1

print(f"Stored {len(vis)} snapshots in {run_dir}")

# ----------------- Write per-run metadata CSV -----------------
meta_path = os.path.join(run_dir, "snapshots_meta.csv")
with open(meta_path, "w", newline="") as f:
    fieldnames = [
        "snapshot_idx",
        "snapshot_file",
        "sweep_label",
        "L",
        "T",
        "J",
        "h",
        "n_equil_sweeps",
        "n_meas_sweeps",
        "store_every",
    ]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(meta_rows)

print(f"Metadata written to {meta_path}")

# ----------------- Plotting -----------------
# Plot last snapshot
vis.plot_snapshot(snapshot_index=len(vis) - 1, downsample=1)

# Animation over all snapshots in this run
vis.animate_snapshots(indices=None, downsample=1)


Run directory: runs\run_L16_T4.500_J2.000_h0.000_eq100_meas500_freq50
Stored 10 snapshots in runs\run_L16_T4.500_J2.000_h0.000_eq100_meas500_freq50
Metadata written to runs\run_L16_T4.500_J2.000_h0.000_eq100_meas500_freq50\snapshots_meta.csv
